In [27]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from category_encoders import OneHotEncoder, OrdinalEncoder
import pickle

In [29]:
df1 = pd.read_csv('track_df.csv') #read csv
df1.head()

,Unnamed: 0,track_name,track_id,track_popularity,track_number,track_season
0,0,Unfair,78bSWIqoNPdJB8QpCVkv0K,49,1,winter
1,1,The First Snow,2VNpT9fRk1kyksT0S4coZR,52,5,winter
2,2,Love Again,4dYODiAYvJHWQJtNganYCY,59,7,winter
3,3,Let’s Love,0wk4GYAryLcXEMvfhzU1bv,49,2,winter
4,4,너를 사랑하고 있어,2KhNgn4w8LHxn3ybLoofh4,55,1,winter


In [30]:
df2 = pd.read_csv('df_audio_features.csv') #read csv
df2.head()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0,0.668,0.877,5,-3.240,0,0.0364,0.08790,0.000000,0.1540,0.781,97.984,78bSWIqoNPdJB8QpCVkv0K
1,1,0.636,0.674,7,-6.623,0,0.0444,0.00542,0.000000,0.1450,0.814,82.985,2VNpT9fRk1kyksT0S4coZR
2,2,0.684,0.595,4,-5.423,0,0.0364,0.28600,0.000009,0.1060,0.564,95.059,4dYODiAYvJHWQJtNganYCY
3,3,0.542,0.740,6,-2.635,0,0.0316,0.01770,0.000000,0.2670,0.336,133.901,0wk4GYAryLcXEMvfhzU1bv
4,4,0.606,0.460,4,-9.171,1,0.0402,0.85300,0.000004,0.0933,0.131,124.117,2KhNgn4w8LHxn3ybLoofh4


In [31]:
df = df1.merge(df2)

In [32]:
df

,Unnamed: 0,track_name,track_id,track_popularity,track_number,track_season,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id
0,0,Unfair,78bSWIqoNPdJB8QpCVkv0K,49,1,winter,0.668,0.877,5,-3.240,0,0.0364,0.08790,0.000000,0.1540,0.781,97.984,78bSWIqoNPdJB8QpCVkv0K
1,1,The First Snow,2VNpT9fRk1kyksT0S4coZR,52,5,winter,0.636,0.674,7,-6.623,0,0.0444,0.00542,0.000000,0.1450,0.814,82.985,2VNpT9fRk1kyksT0S4coZR
2,2,Love Again,4dYODiAYvJHWQJtNganYCY,59,7,winter,0.684,0.595,4,-5.423,0,0.0364,0.28600,0.000009,0.1060,0.564,95.059,4dYODiAYvJHWQJtNganYCY
3,3,Let’s Love,0wk4GYAryLcXEMvfhzU1bv,49,2,winter,0.542,0.740,6,-2.635,0,0.0316,0.01770,0.000000,0.2670,0.336,133.901,0wk4GYAryLcXEMvfhzU1bv
4,4,너를 사랑하고 있어,2KhNgn4w8LHxn3ybLoofh4,55,1,winter,0.606,0.460,4,-9.171,1,0.0402,0.85300,0.000004,0.0933,0.131,124.117,2KhNgn4w8LHxn3ybLoofh4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,375,Every End of the Day,5m2tbM2w8mG76uwFgla2iF,48,2,spring,0.650,0.790,10,-4.482,1,0.0294,0.08130,0.000000,0.3110,0.387,108.939,5m2tbM2w8mG76uwFgla2iF
376,376,Sunday Monday,1dpixaJ9toPeLL2AXm86ox,0,1,spring,0.751,0.681,5,-3.042,0,0.0279,0.50000,0.000000,0.2180,0.822,95.012,1dpixaJ9toPeLL2AXm86ox
377,377,Thank you,45H0YKiZmzZGE9jiEi7FBq,21,1,spring,0.885,0.426,11,-5.473,0,0.0560,0.08880,0.000010,0.1250,0.871,111.972,45H0YKiZmzZGE9jiEi7FBq
378,378,I think I'm in love,395vU3XBsgKd5iRXT9x94P,28,1,spring,0.744,0.840,8,-2.814,1,0.1940,0.42300,0.000000,0.1080,0.596,117.041,395vU3XBsgKd5iRXT9x94P


In [33]:
df= df.drop(['Unnamed: 0','id'],axis=1)
df

,track_name,track_id,track_popularity,track_number,track_season,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Unfair,78bSWIqoNPdJB8QpCVkv0K,49,1,winter,0.668,0.877,5,-3.240,0,0.0364,0.08790,0.000000,0.1540,0.781,97.984
1,The First Snow,2VNpT9fRk1kyksT0S4coZR,52,5,winter,0.636,0.674,7,-6.623,0,0.0444,0.00542,0.000000,0.1450,0.814,82.985
2,Love Again,4dYODiAYvJHWQJtNganYCY,59,7,winter,0.684,0.595,4,-5.423,0,0.0364,0.28600,0.000009,0.1060,0.564,95.059
3,Let’s Love,0wk4GYAryLcXEMvfhzU1bv,49,2,winter,0.542,0.740,6,-2.635,0,0.0316,0.01770,0.000000,0.2670,0.336,133.901
4,너를 사랑하고 있어,2KhNgn4w8LHxn3ybLoofh4,55,1,winter,0.606,0.460,4,-9.171,1,0.0402,0.85300,0.000004,0.0933,0.131,124.117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Every End of the Day,5m2tbM2w8mG76uwFgla2iF,48,2,spring,0.650,0.790,10,-4.482,1,0.0294,0.08130,0.000000,0.3110,0.387,108.939
376,Sunday Monday,1dpixaJ9toPeLL2AXm86ox,0,1,spring,0.751,0.681,5,-3.042,0,0.0279,0.50000,0.000000,0.2180,0.822,95.012
377,Thank you,45H0YKiZmzZGE9jiEi7FBq,21,1,spring,0.885,0.426,11,-5.473,0,0.0560,0.08880,0.000010,0.1250,0.871,111.972
378,I think I'm in love,395vU3XBsgKd5iRXT9x94P,28,1,spring,0.744,0.840,8,-2.814,1,0.1940,0.42300,0.000000,0.1080,0.596,117.041


In [34]:
train, test = train_test_split(df, test_size=0.25, random_state=42)
print('원래 데이터 : {}'.format(df.shape))
print('훈련 데이터 : {}'.format(train.shape))
print('테스트 데이터 : {}'.format(test.shape))

원래 데이터 : (380, 16)
훈련 데이터 : (285, 16)
테스트 데이터 : (95, 16)


In [35]:
target = 'track_season'
features = df.columns.drop(target)

X_train =  train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

print('X_train : {}'.format(X_train.shape))
print('y_train : {}'.format(y_train.shape))
print('\nX_test : {}'.format(X_test.shape))
print('y_test : {}'.format(y_test.shape))

X_train : (285, 15)
y_train : (285,)

X_test : (95, 15)
y_test : (95,)


In [36]:
model = make_pipeline(
    OneHotEncoder(),
    MinMaxScaler(),
    XGBClassifier()
    )

In [37]:
model.fit(X_train,y_train)

[15:03:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\cm\anaconda3\envs\section3_pj\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['track_name', 'track_id'])),
                ('minmaxscaler', MinMaxScaler()),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, enable_categorical=False,
                               gamma=0, gpu_id=-1, importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=12, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=0, reg_alpha=0, reg_lambda=1,
                           

In [38]:
from sklearn.metrics import classification_report

y_pred_test = model.predict(X_test)

print("검증 정확도", model.score(X_test, y_test))

print(
    classification_report(
        y_test, y_pred_test
    )
)

검증 정확도 0.5684210526315789
              precision    recall  f1-score   support

      autumn       0.41      0.56      0.47        16
      spring       0.54      0.61      0.57        23
      summer       0.69      0.87      0.77        23
      winter       0.61      0.33      0.43        33

    accuracy                           0.57        95
   macro avg       0.56      0.59      0.56        95
weighted avg       0.58      0.57      0.55        95



In [39]:
import pickle

with open('model.pkl','wb') as pickle_file:
    pickle.dump(model, pickle_file)